In [ ]:
###########################전체용############################

In [8]:
import bs4
import requests
from urllib import parse
import selenium
import csv
import re

#함수 사용에 앞서서 반드시 driver = webdriver.Chrome("C:/chromedriver.exe") 등의 드라이버를 실행시킬 것

def munpia_crawling(search_name):

    tmp = re.split(':|\(|-', search_name)[0]
    print(tmp)
    search_name_nonspace=search_name.replace(" ","")

    url = "https://novel.munpia.com/page/hd.platinum/view/search/keyword/" + parse.quote(tmp) + "/order/search_result"


    r=requests.get(url)
    c=r.content
    html = bs4.BeautifulSoup(c, "html.parser")

    article_wrap=html.find('div',{'class','article_wrap'})
    i=0
    while True:
        article=article_wrap.findAll('div',{'class','article'})[i]
        dl=article.find('dl',{'class','detail'})
        a=dl.find('a')
        title=a.text
        
        i+=1
        if(title in '[E]'):
            continue
        if(title in search_name):
            href=a.attrs['href']
            break

    # initialize
    info_lists = []
    
    url = href

    r=requests.get(url)
    c=r.content
    html = bs4.BeautifulSoup(c, "html.parser")

    print("\n")

    intro=html.find('p',{'class','story'}).text
    intro=re.sub('^ [ 0-9a-zA-Zㄱ-ㅣ가-힣]','',intro)
    print("작품소개\n" + "----------------------------------------\n" + intro + "\n----------------------------------------")

    cover_box=html.find('div',{'class','cover-box'})
    img_src=cover_box.find('img').attrs['src']
    print("이미지 : " + img_src)

    box=html.find('div',{'class','dd detail-box'})
    h2=box.find('h2')
    exclusive=h2.find('span')
    if(exclusive==None):
        #exclusive='미독점'
        #print("독점여부 : "+ exclusive)
        title=h2.text.replace('\n',"")
        print("제목 : " + title)
        #print("제목 : " + title,end='')
    else:
        #exclusive=exclusive.text
        #print("독점여부 : " + exclusive)
        title=h2.text.split("독점\n")[1].replace('\n','').replace(u'\xa0','')
        print("제목 : " + title)

    p=box.find('p',{'class','meta-path'})
    genre=p.find('strong').text
    if ',' in genre:
        genre=genre.split(',')[0] + ", " + genre.split(',')[1]
        print("장르 : " + genre)
    else:
        print("장르 : " + genre)

    #fr=box.find('div',{'class','fr'})
    #prefer=fr.find('b').text
    #print("선호수 : " + prefer)
    """
    days = dict.fromkeys(["mon", "tue", "wed", "thu", "fri", "sat", "sun"], False)
    novel_period=box.find('div',{'class','novel-period'})
    period=""
    if novel_period!=None:
        if(novel_period.find('span',{'class','pluszone-mon'})):
            days.update(dict.fromkeys(["mon"], True))
            period=period+"월,"
        if(novel_period.find('span',{'class','pluszone-tue'})):
            days.update(dict.fromkeys(["tue"], True))
            period=period+"화,"
        if(novel_period.find('span',{'class','pluszone-wed'})):
            days.update(dict.fromkeys(["wed"], True))
            period=period+"수,"
        if(novel_period.find('span',{'class','pluszone-thu'})):
            days.update(dict.fromkeys(["thu"], True))
            period=period+"목,"
        if(novel_period.find('span',{'class','pluszone-fri'})):
            days.update(dict.fromkeys(["fri"], True))
            period=period+"금,"
        if(novel_period.find('span',{'class','pluszone-sat'})):
            days.update(dict.fromkeys(["sat"], True))
            period=period+"토,"
        if(novel_period.find('span',{'class','pluszone-sun'})):
            days.update(dict.fromkeys(["sun"], True))
            period=period+"일,"


        if(all(value==True for value in days.values())==True):
            period="매일 "

        print("연재 주기 : " + period[:-1] + " 연재")



    else:
        print("연재 주기 : 비주기적 연재")
    """

    age=box.find({'class','xui-icon xui-adult'})
    if(age==None):
        age='전체이용가'
        print("연령등급 : "+age)
    else:
        age=age.text
        print("연령등급 : " + age)


    iconset=box.find('p',{'class','iconset'})
    complete=iconset.find('span',{'class','xui-icon xui-finish'})
    if complete!=None:
        complete=complete.text
        print("완결여부 : " + complete)
    else:
        complete='미완결'
        print("완결여부 : "+ complete)

    writer_dl=box.find('dl',{'class','meta-author meta'})
    author=writer_dl.find('dd').text
    print("작가 : " + author)

    etc_dl=box.findAll('dl',{'class','meta-etc meta'})
    days_dl=etc_dl[0]
    days_dd=days_dl.findAll('dd')
    pub_date=days_dd[0].text
    print("작품등록일 : " + pub_date)
    #recently_update=days_dd[1].text
    #print("최근 업데이트일 : " + recently_update)

    number_dl=etc_dl[1]
    number_dd=number_dl.findAll('dd')

    total=number_dd[0].text
    print("총 연재수 : " + total)

    publisher="정보 없음"
    print("출판사: " + publisher)
    #recommend=number_dd[1].text
    #print("추천수 : " + recommend)
    #enjoyer=number_dd[2].text
    #print("조회수 : " + enjoyer)
    
    info_list = [title.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 genre.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 author.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 complete.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 img_src.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 age.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 total.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 pub_date.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 publisher.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 intro.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 href.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6','')
                ]                  
    return info_list


def munpia_crawling2(search_name):

    tmp = re.split(':|\(|-', search_name)[0]
    print(tmp)
    search_name_nonspace=search_name.replace(" ","")

    url = "https://novel.munpia.com/page/hd.platinum/view/search/keyword/" + parse.quote(tmp) + "/order/search_result"

    r=requests.get(url)
    c=r.content
    html = bs4.BeautifulSoup(c, "html.parser")

    article_wrap=html.find('div',{'class','article_wrap'})
    i=0
    while True:
        article=article_wrap.findAll('div',{'class','article'})[i]
        dl=article.find('dl',{'class','detail'})
        a=dl.find('a')
        title=a.text
        
        i+=1
        if(title in '[E]'):
            continue
        if(title in search_name):
            href=a.attrs['href']
            break

    # initialize
    info_lists = []
    
    url = href

    r=requests.get(url)
    c=r.content
    html = bs4.BeautifulSoup(c, "html.parser")

    print("\n")

    intro=""
    print("작품소개\n" + "----------------------------------------\n" + intro + "\n----------------------------------------")

    cover_box=html.find('div',{'class','cover-box'})
    img_src=cover_box.find('img').attrs['src']
    print("이미지 : " + img_src)

    box=html.find('div',{'class','dd detail-box'})
    h2=box.find('h2')
    exclusive=h2.find('span')
    if(exclusive==None):
        #exclusive='미독점'
        #print("독점여부 : "+ exclusive)
        title=h2.text.replace('\n',"")
        print("제목 : " + title)
        #print("제목 : " + title,end='')
    else:
        #exclusive=exclusive.text
        #print("독점여부 : " + exclusive)
        title=h2.text.split("독점\n")[1].replace('\n','').replace(u'\xa0','')
        print("제목 : " + title)

    p=box.find('p',{'class','meta-path'})
    genre=p.find('strong').text
    if ',' in genre:
        genre=genre.split(',')[0] + ", " + genre.split(',')[1]
        print("장르 : " + genre)
    else:
        print("장르 : " + genre)

    #fr=box.find('div',{'class','fr'})
    #prefer=fr.find('b').text
    #print("선호수 : " + prefer)
    """
    days = dict.fromkeys(["mon", "tue", "wed", "thu", "fri", "sat", "sun"], False)
    novel_period=box.find('div',{'class','novel-period'})
    period=""
    if novel_period!=None:
        if(novel_period.find('span',{'class','pluszone-mon'})):
            days.update(dict.fromkeys(["mon"], True))
            period=period+"월,"
        if(novel_period.find('span',{'class','pluszone-tue'})):
            days.update(dict.fromkeys(["tue"], True))
            period=period+"화,"
        if(novel_period.find('span',{'class','pluszone-wed'})):
            days.update(dict.fromkeys(["wed"], True))
            period=period+"수,"
        if(novel_period.find('span',{'class','pluszone-thu'})):
            days.update(dict.fromkeys(["thu"], True))
            period=period+"목,"
        if(novel_period.find('span',{'class','pluszone-fri'})):
            days.update(dict.fromkeys(["fri"], True))
            period=period+"금,"
        if(novel_period.find('span',{'class','pluszone-sat'})):
            days.update(dict.fromkeys(["sat"], True))
            period=period+"토,"
        if(novel_period.find('span',{'class','pluszone-sun'})):
            days.update(dict.fromkeys(["sun"], True))
            period=period+"일,"


        if(all(value==True for value in days.values())==True):
            period="매일 "

        print("연재 주기 : " + period[:-1] + " 연재")



    else:
        print("연재 주기 : 비주기적 연재")
    """

    age=box.find({'class','xui-icon xui-adult'})
    if(age==None):
        age='전체이용가'
        print("연령등급 : "+age)
    else:
        age=age.text
        print("연령등급 : " + age)


    iconset=box.find('p',{'class','iconset'})
    complete=iconset.find('span',{'class','xui-icon xui-finish'})
    if complete!=None:
        complete=complete.text
        print("완결여부 : " + complete)
    else:
        complete='미완결'
        print("완결여부 : "+ complete)

    writer_dl=box.find('dl',{'class','meta-author meta'})
    author=writer_dl.find('dd').text
    print("작가 : " + author)

    etc_dl=box.findAll('dl',{'class','meta-etc meta'})
    days_dl=etc_dl[0]
    days_dd=days_dl.findAll('dd')
    pub_date=days_dd[0].text
    print("작품등록일 : " + pub_date)
    #recently_update=days_dd[1].text
    #print("최근 업데이트일 : " + recently_update)

    number_dl=etc_dl[1]
    number_dd=number_dl.findAll('dd')

    total=number_dd[0].text
    print("총 연재수 : " + total)

    publisher="정보 없음"
    print("출판사: " + publisher)
    #recommend=number_dd[1].text
    #print("추천수 : " + recommend)
    #enjoyer=number_dd[2].text
    #print("조회수 : " + enjoyer)
    
    info_list = [title.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 genre.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 author.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 complete.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 img_src.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 age.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 total.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 pub_date.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 publisher.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 intro.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 href.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6','')
                ]                   
    
    return info_list

"""
driver = webdriver.Chrome("C:/chromedriver.exe")
url="https://mm.munpia.com/?menu=member&action=login"
driver.get(url)

time.sleep(2)
elem_login = driver.find_element_by_xpath('//*[@id="member_id"]') # 현재 크롬에 떠 있는 웹페이지에서 id 속성 값이 id 인 element를 찾음
elem_login.clear()
elem_login.send_keys("narukane") # typing

elem_login = driver.find_element_by_xpath('//*[@id="member_pw"]')
elem_login.clear()
elem_login.send_keys("ksj3721*")

elem_enter=driver.find_element_by_xpath('//*[@id="sub_contents"]/article/div/div[1]/div[4]/form/table/tbody/tr[4]/td/input')
elem_enter.click()
"""

f = open('munpia_title_list.csv','r', encoding='cp949')
f_w = open('output.csv', 'a', encoding='cp949', newline='')
wr = csv.writer(f_w)
csvReader = csv.reader(f)


# 첫 헤더 제외
next(csvReader)

for row in csvReader:
    try:
        wr.writerow(munpia_crawling(row[0]))
    except:
        wr.writerow(munpia_crawling2(row[0]))

f_w.close()
f.close()


"""
driver = webdriver.Chrome("C:/chromedriver.exe")


munpia_crawling('1983')
info_lists.append(info_list)

df = pd.DataFrame(info_lists, columns = ["title", "genre", "author", "complete", "img_src", "age", "total", "pub_date", "publisher", "intro","href"])

# csv 파일 만들기
df.to_csv("{}.csv".format(title), index=False, encoding='cp949')
print("저장 완료.\n")
#Munpia_Crawling.munpia_crawling('1983')
"""

재벌가 대물사위
재벌가 대물사위


AttributeError: 'NoneType' object has no attribute 'find'

In [11]:
import bs4
import requests
from urllib import parse
import selenium
import csv
import re
import warnings # 경고메세지 제거

#함수 사용에 앞서서 반드시 driver = webdriver.Chrome("C:/chromedriver.exe") 등의 드라이버를 실행시킬 것

def munpia_crawling(href):

    # initialize
    info_lists = []
    
    url = href

    r=requests.get(url)
    c=r.content
    html = bs4.BeautifulSoup(c, "html.parser")

    print("\n")

    intro=html.find('p',{'class','story'}).text
    intro=re.sub('^ [ 0-9a-zA-Zㄱ-ㅣ가-힣]','',intro)
    print("작품소개\n" + "----------------------------------------\n" + intro + "\n----------------------------------------")

    cover_box=html.find('div',{'class','cover-box'})
    img_src=cover_box.find('img').attrs['src']
    print("이미지 : " + img_src)

    box=html.find('div',{'class','dd detail-box'})
    h2=box.find('h2')
    exclusive=h2.find('span')
    if(exclusive==None):
        #exclusive='미독점'
        #print("독점여부 : "+ exclusive)
        title=h2.text.replace('\n',"")
        print("제목 : " + title)
        #print("제목 : " + title,end='')
    else:
        #exclusive=exclusive.text
        #print("독점여부 : " + exclusive)
        title=h2.text.split("독점\n")[1].replace('\n','').replace(u'\xa0','')
        print("제목 : " + title)

    p=box.find('p',{'class','meta-path'})
    genre=p.find('strong').text
    if ',' in genre:
        genre=genre.split(',')[0] + ", " + genre.split(',')[1]
        print("장르 : " + genre)
    else:
        print("장르 : " + genre)

    #fr=box.find('div',{'class','fr'})
    #prefer=fr.find('b').text
    #print("선호수 : " + prefer)
    """
    days = dict.fromkeys(["mon", "tue", "wed", "thu", "fri", "sat", "sun"], False)
    novel_period=box.find('div',{'class','novel-period'})
    period=""
    if novel_period!=None:
        if(novel_period.find('span',{'class','pluszone-mon'})):
            days.update(dict.fromkeys(["mon"], True))
            period=period+"월,"
        if(novel_period.find('span',{'class','pluszone-tue'})):
            days.update(dict.fromkeys(["tue"], True))
            period=period+"화,"
        if(novel_period.find('span',{'class','pluszone-wed'})):
            days.update(dict.fromkeys(["wed"], True))
            period=period+"수,"
        if(novel_period.find('span',{'class','pluszone-thu'})):
            days.update(dict.fromkeys(["thu"], True))
            period=period+"목,"
        if(novel_period.find('span',{'class','pluszone-fri'})):
            days.update(dict.fromkeys(["fri"], True))
            period=period+"금,"
        if(novel_period.find('span',{'class','pluszone-sat'})):
            days.update(dict.fromkeys(["sat"], True))
            period=period+"토,"
        if(novel_period.find('span',{'class','pluszone-sun'})):
            days.update(dict.fromkeys(["sun"], True))
            period=period+"일,"


        if(all(value==True for value in days.values())==True):
            period="매일 "

        print("연재 주기 : " + period[:-1] + " 연재")



    else:
        print("연재 주기 : 비주기적 연재")
    """

    age=box.find({'class','xui-icon xui-adult'})
    if(age==None):
        age='전체이용가'
        print("연령등급 : "+age)
    else:
        age=age.text
        print("연령등급 : " + age)


    iconset=box.find('p',{'class','iconset'})
    complete=iconset.find('span',{'class','xui-icon xui-finish'})
    if complete!=None:
        complete=complete.text
        print("완결여부 : " + complete)
    else:
        complete='미완결'
        print("완결여부 : "+ complete)

    writer_dl=box.find('dl',{'class','meta-author meta'})
    author=writer_dl.find('dd').text
    print("작가 : " + author)

    etc_dl=box.findAll('dl',{'class','meta-etc meta'})
    days_dl=etc_dl[0]
    days_dd=days_dl.findAll('dd')
    pub_date=days_dd[0].text
    print("작품등록일 : " + pub_date)
    #recently_update=days_dd[1].text
    #print("최근 업데이트일 : " + recently_update)

    number_dl=etc_dl[1]
    number_dd=number_dl.findAll('dd')

    total=number_dd[0].text
    print("총 연재수 : " + total)

    publisher="정보 없음"
    print("출판사: " + publisher)
    #recommend=number_dd[1].text
    #print("추천수 : " + recommend)
    #enjoyer=number_dd[2].text
    #print("조회수 : " + enjoyer)
    
    info_list = [title.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 genre.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 author.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 complete.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 img_src.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 age.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 total.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 pub_date.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 publisher.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 intro.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 href.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6','')
                ]             
    return info_list


def munpia_crawling2(href):


    # initialize
    info_lists = []
    
    url = href

    r=requests.get(url)
    c=r.content
    html = bs4.BeautifulSoup(c, "html.parser")

    print("\n")

    intro=""
    print("작품소개\n" + "----------------------------------------\n" + intro + "\n----------------------------------------")

    cover_box=html.find('div',{'class','cover-box'})
    img_src=cover_box.find('img').attrs['src']
    print("이미지 : " + img_src)

    box=html.find('div',{'class','dd detail-box'})
    h2=box.find('h2')
    exclusive=h2.find('span')
    if(exclusive==None):
        #exclusive='미독점'
        #print("독점여부 : "+ exclusive)
        title=h2.text.replace('\n',"")
        print("제목 : " + title)
        #print("제목 : " + title,end='')
    else:
        #exclusive=exclusive.text
        #print("독점여부 : " + exclusive)
        title=h2.text.split("독점\n")[1].replace('\n','').replace(u'\xa0','')
        print("제목 : " + title)

    p=box.find('p',{'class','meta-path'})
    genre=p.find('strong').text
    if ',' in genre:
        genre=genre.split(',')[0] + ", " + genre.split(',')[1]
        print("장르 : " + genre)
    else:
        print("장르 : " + genre)

    #fr=box.find('div',{'class','fr'})
    #prefer=fr.find('b').text
    #print("선호수 : " + prefer)
    """
    days = dict.fromkeys(["mon", "tue", "wed", "thu", "fri", "sat", "sun"], False)
    novel_period=box.find('div',{'class','novel-period'})
    period=""
    if novel_period!=None:
        if(novel_period.find('span',{'class','pluszone-mon'})):
            days.update(dict.fromkeys(["mon"], True))
            period=period+"월,"
        if(novel_period.find('span',{'class','pluszone-tue'})):
            days.update(dict.fromkeys(["tue"], True))
            period=period+"화,"
        if(novel_period.find('span',{'class','pluszone-wed'})):
            days.update(dict.fromkeys(["wed"], True))
            period=period+"수,"
        if(novel_period.find('span',{'class','pluszone-thu'})):
            days.update(dict.fromkeys(["thu"], True))
            period=period+"목,"
        if(novel_period.find('span',{'class','pluszone-fri'})):
            days.update(dict.fromkeys(["fri"], True))
            period=period+"금,"
        if(novel_period.find('span',{'class','pluszone-sat'})):
            days.update(dict.fromkeys(["sat"], True))
            period=period+"토,"
        if(novel_period.find('span',{'class','pluszone-sun'})):
            days.update(dict.fromkeys(["sun"], True))
            period=period+"일,"


        if(all(value==True for value in days.values())==True):
            period="매일 "

        print("연재 주기 : " + period[:-1] + " 연재")



    else:
        print("연재 주기 : 비주기적 연재")
    """

    age=box.find({'class','xui-icon xui-adult'})
    if(age==None):
        age='전체이용가'
        print("연령등급 : "+age)
    else:
        age=age.text
        print("연령등급 : " + age)


    iconset=box.find('p',{'class','iconset'})
    complete=iconset.find('span',{'class','xui-icon xui-finish'})
    if complete!=None:
        complete=complete.text
        print("완결여부 : " + complete)
    else:
        complete='미완결'
        print("완결여부 : "+ complete)

    writer_dl=box.find('dl',{'class','meta-author meta'})
    author=writer_dl.find('dd').text
    print("작가 : " + author)

    etc_dl=box.findAll('dl',{'class','meta-etc meta'})
    days_dl=etc_dl[0]
    days_dd=days_dl.findAll('dd')
    pub_date=days_dd[0].text
    print("작품등록일 : " + pub_date)
    #recently_update=days_dd[1].text
    #print("최근 업데이트일 : " + recently_update)

    number_dl=etc_dl[1]
    number_dd=number_dl.findAll('dd')

    total=number_dd[0].text
    print("총 연재수 : " + total)

    publisher="정보 없음"
    print("출판사: " + publisher)
    #recommend=number_dd[1].text
    #print("추천수 : " + recommend)
    #enjoyer=number_dd[2].text
    #print("조회수 : " + enjoyer)
    
    info_list = [title.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 genre.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 author.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 complete.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 img_src.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 age.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 total.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 pub_date.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 publisher.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 intro.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 href.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6','')
                ]          
    return info_list



f_w = open('output.csv', 'a', encoding='cp949', newline='')
wr = csv.writer(f_w)

try:
    wr.writerow(munpia_crawling('https://novel.munpia.com/10983'))
except:
    wr.writerow(munpia_crawling2('https://novel.munpia.com/10983'))

f_w.close()




작품소개
----------------------------------------
중한 질병을 가지고 있는 대승그룹 사장 오재범은 우연히 대승 사이트에 들어와 대승의 모든것을 알아 차린 신동한을 없애라고 명령한다. 그러나 자기의 죽음을 직감한 그는 신동한의 육체에 자기의 기억인자를 삽입하는 방법으로 새로운 육체를 얻어 살인무기를 완성한다. 이와 때를 같이 해 실수로 여비서가 죽자 그녀 또한 강한 무기로 탄생시킨다. 이 무기들의 존재를 알아차린 FBI,빈 라덴, 신 나치주의...모두가 무기를 빼앗기 위해 혈안이 되어있고, 한국정부 또한 무기를 손에 넣고자 오재범의 기술원을 점령한다. 그러나, 오 재범이 무기를 생산한 이유는 따로 있는데...
----------------------------------------
이미지 : //cdn1.munpia.com/files/attach/2013/0817/000/MK2G9U5vzhlakXsX.jpg
제목 : K-프로젝트
장르 : 전쟁·밀리터리
연령등급 : 전체이용가
완결여부 : 완결
작가 : 
돈까밀로


작품등록일 : 2013.06.09 09:16
총 연재수 : 50 회
출판사: 정보 없음


In [ ]:
###########################성인용############################

In [6]:
import bs4
import requests
from urllib import parse
import selenium
import csv
import re
from selenium import webdriver
import warnings # 경고메세지 제거
import time

#함수 사용에 앞서서 반드시 driver = webdriver.Chrome("C:/chromedriver.exe") 등의 드라이버를 실행시킬 것

def munpia_adult_crawling(search_name):

    tmp = re.split(':|\(|-', search_name)[0]
    print(tmp)
    search_name_nonspace=search_name.replace(" ","")

    url = "https://novel.munpia.com/page/hd.platinum/view/search/keyword/" + parse.quote(tmp) + "/order/search_result"


    driver.get(url)
    html = bs4.BeautifulSoup(driver.page_source, "html.parser")

    article_wrap=html.find('div',{'class','article_wrap'})
    i=0
    while True:
        article=article_wrap.findAll('div',{'class','article'})[i]
        dl=article.find('dl',{'class','detail'})
        a=dl.find('a')
        title=a.text
        
        i+=1
        if(title in '[E]'):
            continue
        if(title in search_name):
            href=a.attrs['href']
            break

    # initialize
    info_lists = []
    
    url = href

    driver.get(url)
    html = bs4.BeautifulSoup(driver.page_source, "html.parser")

    print("\n")

    intro=html.find('p',{'class','story'}).text
    intro=re.sub('^ [ 0-9a-zA-Zㄱ-ㅣ가-힣]','',intro)
    print("작품소개\n" + "----------------------------------------\n" + intro + "\n----------------------------------------")

    cover_box=html.find('div',{'class','cover-box'})
    img_src=cover_box.find('img').attrs['src']
    print("이미지 : " + img_src)

    box=html.find('div',{'class','dd detail-box'})
    h2=box.find('h2')
    exclusive=h2.find('span')
    if(exclusive==None):
        #exclusive='미독점'
        #print("독점여부 : "+ exclusive)
        title=h2.text.replace('\n',"")
        print("제목 : " + title)
        #print("제목 : " + title,end='')
    else:
        #exclusive=exclusive.text
        #print("독점여부 : " + exclusive)
        title=h2.text.split("독점\n")[1].replace('\n','').replace(u'\xa0','')
        print("제목 : " + title)

    p=box.find('p',{'class','meta-path'})
    genre=p.find('strong').text
    if ',' in genre:
        genre=genre.split(',')[0] + ", " + genre.split(',')[1]
        print("장르 : " + genre)
    else:
        print("장르 : " + genre)

    #fr=box.find('div',{'class','fr'})
    #prefer=fr.find('b').text
    #print("선호수 : " + prefer)
    """
    days = dict.fromkeys(["mon", "tue", "wed", "thu", "fri", "sat", "sun"], False)
    novel_period=box.find('div',{'class','novel-period'})
    period=""
    if novel_period!=None:
        if(novel_period.find('span',{'class','pluszone-mon'})):
            days.update(dict.fromkeys(["mon"], True))
            period=period+"월,"
        if(novel_period.find('span',{'class','pluszone-tue'})):
            days.update(dict.fromkeys(["tue"], True))
            period=period+"화,"
        if(novel_period.find('span',{'class','pluszone-wed'})):
            days.update(dict.fromkeys(["wed"], True))
            period=period+"수,"
        if(novel_period.find('span',{'class','pluszone-thu'})):
            days.update(dict.fromkeys(["thu"], True))
            period=period+"목,"
        if(novel_period.find('span',{'class','pluszone-fri'})):
            days.update(dict.fromkeys(["fri"], True))
            period=period+"금,"
        if(novel_period.find('span',{'class','pluszone-sat'})):
            days.update(dict.fromkeys(["sat"], True))
            period=period+"토,"
        if(novel_period.find('span',{'class','pluszone-sun'})):
            days.update(dict.fromkeys(["sun"], True))
            period=period+"일,"


        if(all(value==True for value in days.values())==True):
            period="매일 "

        print("연재 주기 : " + period[:-1] + " 연재")



    else:
        print("연재 주기 : 비주기적 연재")
    """

    age=box.find({'class','xui-icon xui-adult'})
    if(age==None):
        age='19세'
        print("연령등급 : "+age)
    else:
        age=age.text
        print("연령등급 : " + age)


    iconset=box.find('p',{'class','iconset'})
    complete=iconset.find('span',{'class','xui-icon xui-finish'})
    if complete!=None:
        complete=complete.text
        print("완결여부 : " + complete)
    else:
        complete='미완결'
        print("완결여부 : "+ complete)

    writer_dl=box.find('dl',{'class','meta-author meta'})
    author=writer_dl.find('dd').text
    print("작가 : " + author)

    etc_dl=box.findAll('dl',{'class','meta-etc meta'})
    days_dl=etc_dl[0]
    days_dd=days_dl.findAll('dd')
    pub_date=days_dd[0].text
    print("작품등록일 : " + pub_date)
    #recently_update=days_dd[1].text
    #print("최근 업데이트일 : " + recently_update)

    number_dl=etc_dl[1]
    number_dd=number_dl.findAll('dd')

    total=number_dd[0].text
    print("총 연재수 : " + total)

    publisher="정보 없음"
    print("출판사: " + publisher)
    #recommend=number_dd[1].text
    #print("추천수 : " + recommend)
    #enjoyer=number_dd[2].text
    #print("조회수 : " + enjoyer)
    
    info_list = [title.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 genre.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 author.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 complete.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 img_src.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 age.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 total.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 pub_date.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 publisher.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 intro.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 href.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6','')
                ]                  
    return info_list


def munpia_adult_crawling2(search_name):

    tmp = re.split(':|\(|-', search_name)[0]
    print(tmp)
    search_name_nonspace=search_name.replace(" ","")

    url = "https://novel.munpia.com/page/hd.platinum/view/search/keyword/" + parse.quote(tmp) + "/order/search_result"

    driver.get(url)
    html = bs4.BeautifulSoup(driver.page_source, "html.parser")

    article_wrap=html.find('div',{'class','article_wrap'})
    i=0
    while True:
        article=article_wrap.findAll('div',{'class','article'})[i]
        dl=article.find('dl',{'class','detail'})
        a=dl.find('a')
        title=a.text
        
        i+=1
        if(title in '[E]'):
            continue
        if(title in search_name):
            href=a.attrs['href']
            break

    # initialize
    info_lists = []
    
    url = href

    driver.get(url)
    html = bs4.BeautifulSoup(driver.page_source, "html.parser")

    print("\n")

    intro=""
    print("작품소개\n" + "----------------------------------------\n" + intro + "\n----------------------------------------")

    cover_box=html.find('div',{'class','cover-box'})
    img_src=cover_box.find('img').attrs['src']
    print("이미지 : " + img_src)

    box=html.find('div',{'class','dd detail-box'})
    h2=box.find('h2')
    exclusive=h2.find('span')
    if(exclusive==None):
        #exclusive='미독점'
        #print("독점여부 : "+ exclusive)
        title=h2.text.replace('\n',"")
        print("제목 : " + title)
        #print("제목 : " + title,end='')
    else:
        #exclusive=exclusive.text
        #print("독점여부 : " + exclusive)
        title=h2.text.split("독점\n")[1].replace('\n','').replace(u'\xa0','')
        print("제목 : " + title)

    p=box.find('p',{'class','meta-path'})
    genre=p.find('strong').text
    if ',' in genre:
        genre=genre.split(',')[0] + ", " + genre.split(',')[1]
        print("장르 : " + genre)
    else:
        print("장르 : " + genre)

    #fr=box.find('div',{'class','fr'})
    #prefer=fr.find('b').text
    #print("선호수 : " + prefer)
    """
    days = dict.fromkeys(["mon", "tue", "wed", "thu", "fri", "sat", "sun"], False)
    novel_period=box.find('div',{'class','novel-period'})
    period=""
    if novel_period!=None:
        if(novel_period.find('span',{'class','pluszone-mon'})):
            days.update(dict.fromkeys(["mon"], True))
            period=period+"월,"
        if(novel_period.find('span',{'class','pluszone-tue'})):
            days.update(dict.fromkeys(["tue"], True))
            period=period+"화,"
        if(novel_period.find('span',{'class','pluszone-wed'})):
            days.update(dict.fromkeys(["wed"], True))
            period=period+"수,"
        if(novel_period.find('span',{'class','pluszone-thu'})):
            days.update(dict.fromkeys(["thu"], True))
            period=period+"목,"
        if(novel_period.find('span',{'class','pluszone-fri'})):
            days.update(dict.fromkeys(["fri"], True))
            period=period+"금,"
        if(novel_period.find('span',{'class','pluszone-sat'})):
            days.update(dict.fromkeys(["sat"], True))
            period=period+"토,"
        if(novel_period.find('span',{'class','pluszone-sun'})):
            days.update(dict.fromkeys(["sun"], True))
            period=period+"일,"


        if(all(value==True for value in days.values())==True):
            period="매일 "

        print("연재 주기 : " + period[:-1] + " 연재")



    else:
        print("연재 주기 : 비주기적 연재")
    """

    age=box.find({'class','xui-icon xui-adult'})
    if(age==None):
        age='19세'
        print("연령등급 : "+age)
    else:
        age=age.text
        print("연령등급 : " + age)


    iconset=box.find('p',{'class','iconset'})
    complete=iconset.find('span',{'class','xui-icon xui-finish'})
    if complete!=None:
        complete=complete.text
        print("완결여부 : " + complete)
    else:
        complete='미완결'
        print("완결여부 : "+ complete)

    writer_dl=box.find('dl',{'class','meta-author meta'})
    author=writer_dl.find('dd').text
    print("작가 : " + author)

    etc_dl=box.findAll('dl',{'class','meta-etc meta'})
    days_dl=etc_dl[0]
    days_dd=days_dl.findAll('dd')
    pub_date=days_dd[0].text
    print("작품등록일 : " + pub_date)
    #recently_update=days_dd[1].text
    #print("최근 업데이트일 : " + recently_update)

    number_dl=etc_dl[1]
    number_dd=number_dl.findAll('dd')

    total=number_dd[0].text
    print("총 연재수 : " + total)

    publisher="정보 없음"
    print("출판사: " + publisher)
    #recommend=number_dd[1].text
    #print("추천수 : " + recommend)
    #enjoyer=number_dd[2].text
    #print("조회수 : " + enjoyer)
    
    info_list = [title.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 genre.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 author.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 complete.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 img_src.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 age.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 total.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 pub_date.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 publisher.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 intro.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 href.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6','')
                ]                   
    
    return info_list

"""
driver = webdriver.Chrome("C:/chromedriver.exe")
url="https://mm.munpia.com/?menu=member&action=login"
driver.get(url)

time.sleep(2)
elem_login = driver.find_element_by_xpath('//*[@id="member_id"]') # 현재 크롬에 떠 있는 웹페이지에서 id 속성 값이 id 인 element를 찾음
elem_login.clear()
elem_login.send_keys("narukane") # typing

elem_login = driver.find_element_by_xpath('//*[@id="member_pw"]')
elem_login.clear()
elem_login.send_keys("ksj3721*")

elem_enter=driver.find_element_by_xpath('//*[@id="sub_contents"]/article/div/div[1]/div[4]/form/table/tbody/tr[4]/td/input')
elem_enter.click()
"""
warnings.filterwarnings(action='ignore') # 경고 메세지 제거

# 로그인 계정 정보 [ID : dawaplease / PWD : dawa20152017]
ig_id = 'narukane'
ig_pwd = 'ksj3721*'

# 크롬드라이버 호출
driver = webdriver.Chrome("C:/chromedriver.exe")

# 로그인 [ID : dawaplease / PW : dawa20152017]
driver.get('https://www.munpia.com/')

time.sleep(2)

id_input = driver.find_element_by_xpath('/html/body/div[2]/div[1]/aside/section[1]/form/div[1]/div[1]/div/input[1]')
id_input.send_keys(ig_id)
password_input = driver.find_element_by_xpath('/html/body/div[2]/div[1]/aside/section[1]/form/div[1]/div[1]/div/input[2]')
password_input.send_keys(ig_pwd)

driver.find_element_by_xpath('//*[@id="FORM-LOGIN"]/div[1]/div[2]').click()

time.sleep(2)


f = open('munpia_title_adult_list.csv','r', encoding='cp949')
f_w = open('output.csv', 'a', encoding='cp949', newline='')
wr = csv.writer(f_w)
csvReader = csv.reader(f)


# 첫 헤더 제외
#next(csvReader)

for row in csvReader:
    try:
        wr.writerow(munpia_adult_crawling(row[0]))
    except:
        wr.writerow(munpia_adult_crawling2(row[0]))

f_w.close()
f.close()


"""
driver = webdriver.Chrome("C:/chromedriver.exe")


munpia_crawling('1983')
info_lists.append(info_list)

df = pd.DataFrame(info_lists, columns = ["title", "genre", "author", "complete", "img_src", "age", "total", "pub_date", "publisher", "intro","href"])

# csv 파일 만들기
df.to_csv("{}.csv".format(title), index=False, encoding='cp949')
print("저장 완료.\n")
#Munpia_Crawling.munpia_crawling('1983')
"""

비혼라이프


작품소개
----------------------------------------
모태솔로의 주인공에게 어느날 기가막히게 예쁘고 섹시한 요정이 나타난다.
이제까지 불쌍하게 살아왔던 세월을 보상해주듯 말이다.
그리고 그에게 특별한 경험을 하도록 돕겠다고 한다.
그것이 인류의 파멸을 막는 길이라면서...
주인공과 함께 남자들만의 판타지에 빠져 보자

[여성구독 금지] [하렘] [수위높음]
----------------------------------------
이미지 : //cdn1.munpia.com/files/attach/2018/0822/004/PLnZ6BlctDWRqhcj.jpg
제목 : 비혼라이프
장르 : 로맨스,  현대판타지
연령등급 : 19세
완결여부 : 완결
작가 : 
시커먼스


작품등록일 : 2018.09.10 16:00
총 연재수 : 150 회
출판사: 정보 없음
육체의 향기


작품소개
----------------------------------------
한국 남자의 일본 여자정벌기!!
마누라와 이혼을 하고 원룸에서 혼자 살고 있는 이준.
어느날 지하철역 앞에서 반지를 하나 사서 손가락에 끼웠는데....
술에 취해서 일본 AV 동영상을 보면서 딸을 치다가 소원반지에 의해 동영상 속으로 들어간다. 
뜻밖에 일본 AV 배우가 된 이준. 수많은 여배우들과 섹스를 하고 여자를 조종하는 능력까지 생긴다.
한국인들을 경멸하는 일본인들에게 분노하여 일본 여자 만 명에게 깃발을 꽂겠다고 선언한 이준.
차례 차례 일본 여자들을 정복해 간다.
종횡무진하는 그의 활약이 오사카를 무대로 펼쳐진다.
----------------------------------------
이미지 : //cdn1.munpia.com/tpl/novel/core/covers/romance-05.jpg
제목 : 육체의 향기
장르 : 로맨스,  현대판타지
연령등급 : 19세
완결여부 : 완결
작가 : 
이가야


작품등록일 : 2018.12.20 10:00
총 연재수 : 7

'\ndriver = webdriver.Chrome("C:/chromedriver.exe")\n\n\nmunpia_crawling(\'1983\')\ninfo_lists.append(info_list)\n\ndf = pd.DataFrame(info_lists, columns = ["title", "genre", "author", "complete", "img_src", "age", "total", "pub_date", "publisher", "intro","href"])\n\n# csv 파일 만들기\ndf.to_csv("{}.csv".format(title), index=False, encoding=\'cp949\')\nprint("저장 완료.\n")\n#Munpia_Crawling.munpia_crawling(\'1983\')\n'

In [6]:
import bs4
import requests
from urllib import parse
import selenium
import csv
import re
from selenium import webdriver
import warnings # 경고메세지 제거
import time

#함수 사용에 앞서서 반드시 driver = webdriver.Chrome("C:/chromedriver.exe") 등의 드라이버를 실행시킬 것

def munpia_adult_crawling(href):



    # initialize
    info_lists = []
    
    url = href

    driver.get(url)
    html = bs4.BeautifulSoup(driver.page_source, "html.parser")

    print("\n")

    intro=html.find('p',{'class','story'}).text
    intro=re.sub('^ [ 0-9a-zA-Zㄱ-ㅣ가-힣]','',intro)
    print("작품소개\n" + "----------------------------------------\n" + intro + "\n----------------------------------------")

    cover_box=html.find('div',{'class','cover-box'})
    img_src=cover_box.find('img').attrs['src']
    print("이미지 : " + img_src)

    box=html.find('div',{'class','dd detail-box'})
    h2=box.find('h2')
    exclusive=h2.find('span')
    if(exclusive==None):
        #exclusive='미독점'
        #print("독점여부 : "+ exclusive)
        title=h2.text.replace('\n',"")
        print("제목 : " + title)
        #print("제목 : " + title,end='')
    else:
        #exclusive=exclusive.text
        #print("독점여부 : " + exclusive)
        title=h2.text.split("독점\n")[1].replace('\n','').replace(u'\xa0','')
        print("제목 : " + title)

    p=box.find('p',{'class','meta-path'})
    genre=p.find('strong').text
    if ',' in genre:
        genre=genre.split(',')[0] + ", " + genre.split(',')[1]
        print("장르 : " + genre)
    else:
        print("장르 : " + genre)

    #fr=box.find('div',{'class','fr'})
    #prefer=fr.find('b').text
    #print("선호수 : " + prefer)
    """
    days = dict.fromkeys(["mon", "tue", "wed", "thu", "fri", "sat", "sun"], False)
    novel_period=box.find('div',{'class','novel-period'})
    period=""
    if novel_period!=None:
        if(novel_period.find('span',{'class','pluszone-mon'})):
            days.update(dict.fromkeys(["mon"], True))
            period=period+"월,"
        if(novel_period.find('span',{'class','pluszone-tue'})):
            days.update(dict.fromkeys(["tue"], True))
            period=period+"화,"
        if(novel_period.find('span',{'class','pluszone-wed'})):
            days.update(dict.fromkeys(["wed"], True))
            period=period+"수,"
        if(novel_period.find('span',{'class','pluszone-thu'})):
            days.update(dict.fromkeys(["thu"], True))
            period=period+"목,"
        if(novel_period.find('span',{'class','pluszone-fri'})):
            days.update(dict.fromkeys(["fri"], True))
            period=period+"금,"
        if(novel_period.find('span',{'class','pluszone-sat'})):
            days.update(dict.fromkeys(["sat"], True))
            period=period+"토,"
        if(novel_period.find('span',{'class','pluszone-sun'})):
            days.update(dict.fromkeys(["sun"], True))
            period=period+"일,"


        if(all(value==True for value in days.values())==True):
            period="매일 "

        print("연재 주기 : " + period[:-1] + " 연재")



    else:
        print("연재 주기 : 비주기적 연재")
    """

    age=box.find({'class','xui-icon xui-adult'})
    if(age==None):
        age='19세'
        print("연령등급 : "+age)
    else:
        age=age.text
        print("연령등급 : " + age)


    iconset=box.find('p',{'class','iconset'})
    complete=iconset.find('span',{'class','xui-icon xui-finish'})
    if complete!=None:
        complete=complete.text
        print("완결여부 : " + complete)
    else:
        complete='미완결'
        print("완결여부 : "+ complete)

    writer_dl=box.find('dl',{'class','meta-author meta'})
    author=writer_dl.find('dd').text
    print("작가 : " + author)

    etc_dl=box.findAll('dl',{'class','meta-etc meta'})
    days_dl=etc_dl[0]
    days_dd=days_dl.findAll('dd')
    pub_date=days_dd[0].text
    print("작품등록일 : " + pub_date)
    #recently_update=days_dd[1].text
    #print("최근 업데이트일 : " + recently_update)

    number_dl=etc_dl[1]
    number_dd=number_dl.findAll('dd')

    total=number_dd[0].text
    print("총 연재수 : " + total)

    publisher="정보 없음"
    print("출판사: " + publisher)
    #recommend=number_dd[1].text
    #print("추천수 : " + recommend)
    #enjoyer=number_dd[2].text
    #print("조회수 : " + enjoyer)
    
    info_list = [title.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 genre.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 author.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 complete.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 img_src.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 age.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 total.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 pub_date.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 publisher.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 intro.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 href.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6','')
                ]                  
    return info_list


def munpia_adult_crawling2(href):



    # initialize
    info_lists = []
    
    url = href

    driver.get(url)
    html = bs4.BeautifulSoup(driver.page_source, "html.parser")

    print("\n")

    intro=""
    print("작품소개\n" + "----------------------------------------\n" + intro + "\n----------------------------------------")

    cover_box=html.find('div',{'class','cover-box'})
    img_src=cover_box.find('img').attrs['src']
    print("이미지 : " + img_src)

    box=html.find('div',{'class','dd detail-box'})
    h2=box.find('h2')
    exclusive=h2.find('span')
    if(exclusive==None):
        #exclusive='미독점'
        #print("독점여부 : "+ exclusive)
        title=h2.text.replace('\n',"")
        print("제목 : " + title)
        #print("제목 : " + title,end='')
    else:
        #exclusive=exclusive.text
        #print("독점여부 : " + exclusive)
        title=h2.text.split("독점\n")[1].replace('\n','').replace(u'\xa0','')
        print("제목 : " + title)

    p=box.find('p',{'class','meta-path'})
    genre=p.find('strong').text
    if ',' in genre:
        genre=genre.split(',')[0] + ", " + genre.split(',')[1]
        print("장르 : " + genre)
    else:
        print("장르 : " + genre)

    #fr=box.find('div',{'class','fr'})
    #prefer=fr.find('b').text
    #print("선호수 : " + prefer)
    """
    days = dict.fromkeys(["mon", "tue", "wed", "thu", "fri", "sat", "sun"], False)
    novel_period=box.find('div',{'class','novel-period'})
    period=""
    if novel_period!=None:
        if(novel_period.find('span',{'class','pluszone-mon'})):
            days.update(dict.fromkeys(["mon"], True))
            period=period+"월,"
        if(novel_period.find('span',{'class','pluszone-tue'})):
            days.update(dict.fromkeys(["tue"], True))
            period=period+"화,"
        if(novel_period.find('span',{'class','pluszone-wed'})):
            days.update(dict.fromkeys(["wed"], True))
            period=period+"수,"
        if(novel_period.find('span',{'class','pluszone-thu'})):
            days.update(dict.fromkeys(["thu"], True))
            period=period+"목,"
        if(novel_period.find('span',{'class','pluszone-fri'})):
            days.update(dict.fromkeys(["fri"], True))
            period=period+"금,"
        if(novel_period.find('span',{'class','pluszone-sat'})):
            days.update(dict.fromkeys(["sat"], True))
            period=period+"토,"
        if(novel_period.find('span',{'class','pluszone-sun'})):
            days.update(dict.fromkeys(["sun"], True))
            period=period+"일,"


        if(all(value==True for value in days.values())==True):
            period="매일 "

        print("연재 주기 : " + period[:-1] + " 연재")



    else:
        print("연재 주기 : 비주기적 연재")
    """

    age=box.find({'class','xui-icon xui-adult'})
    if(age==None):
        age='19세'
        print("연령등급 : "+age)
    else:
        age=age.text
        print("연령등급 : " + age)


    iconset=box.find('p',{'class','iconset'})
    complete=iconset.find('span',{'class','xui-icon xui-finish'})
    if complete!=None:
        complete=complete.text
        print("완결여부 : " + complete)
    else:
        complete='미완결'
        print("완결여부 : "+ complete)

    writer_dl=box.find('dl',{'class','meta-author meta'})
    author=writer_dl.find('dd').text
    print("작가 : " + author)

    etc_dl=box.findAll('dl',{'class','meta-etc meta'})
    days_dl=etc_dl[0]
    days_dd=days_dl.findAll('dd')
    pub_date=days_dd[0].text
    print("작품등록일 : " + pub_date)
    #recently_update=days_dd[1].text
    #print("최근 업데이트일 : " + recently_update)

    number_dl=etc_dl[1]
    number_dd=number_dl.findAll('dd')

    total=number_dd[0].text
    print("총 연재수 : " + total)

    publisher="정보 없음"
    print("출판사: " + publisher)
    #recommend=number_dd[1].text
    #print("추천수 : " + recommend)
    #enjoyer=number_dd[2].text
    #print("조회수 : " + enjoyer)
    
    info_list = [title.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 genre.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 author.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 complete.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 img_src.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 age.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 total.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 pub_date.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 publisher.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 intro.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 href.replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6','')
                ]                   
    
    return info_list

"""
driver = webdriver.Chrome("C:/chromedriver.exe")
url="https://mm.munpia.com/?menu=member&action=login"
driver.get(url)

time.sleep(2)
elem_login = driver.find_element_by_xpath('//*[@id="member_id"]') # 현재 크롬에 떠 있는 웹페이지에서 id 속성 값이 id 인 element를 찾음
elem_login.clear()
elem_login.send_keys("narukane") # typing

elem_login = driver.find_element_by_xpath('//*[@id="member_pw"]')
elem_login.clear()
elem_login.send_keys("ksj3721*")

elem_enter=driver.find_element_by_xpath('//*[@id="sub_contents"]/article/div/div[1]/div[4]/form/table/tbody/tr[4]/td/input')
elem_enter.click()
"""
warnings.filterwarnings(action='ignore') # 경고 메세지 제거

# 로그인 계정 정보 [ID : dawaplease / PWD : dawa20152017]
ig_id = 'narukane'
ig_pwd = 'ksj3721*'

# 크롬드라이버 호출
driver = webdriver.Chrome("C:/chromedriver.exe")

# 로그인 [ID : dawaplease / PW : dawa20152017]
driver.get('https://www.munpia.com/')

time.sleep(2)

id_input = driver.find_element_by_xpath('/html/body/div[2]/div[1]/aside/section[1]/form/div[1]/div[1]/div/input[1]')
id_input.send_keys(ig_id)
password_input = driver.find_element_by_xpath('/html/body/div[2]/div[1]/aside/section[1]/form/div[1]/div[1]/div/input[2]')
password_input.send_keys(ig_pwd)

driver.find_element_by_xpath('//*[@id="FORM-LOGIN"]/div[1]/div[2]').click()

time.sleep(2)


f = open('munpia_title_adult_list.csv','r', encoding='cp949')
f_w = open('output.csv', 'a', encoding='cp949', newline='')
wr = csv.writer(f_w)
csvReader = csv.reader(f)


# 첫 헤더 제외
#next(csvReader)

try:
    wr.writerow(munpia_adult_crawling('https://novel.munpia.com/46020'))
except:
    wr.writerow(munpia_adult_crawling2('https://novel.munpia.com/46020'))

f_w.close()
f.close()


"""
driver = webdriver.Chrome("C:/chromedriver.exe")


munpia_crawling('1983')
info_lists.append(info_list)

df = pd.DataFrame(info_lists, columns = ["title", "genre", "author", "complete", "img_src", "age", "total", "pub_date", "publisher", "intro","href"])

# csv 파일 만들기
df.to_csv("{}.csv".format(title), index=False, encoding='cp949')
print("저장 완료.\n")
#Munpia_Crawling.munpia_crawling('1983')
"""



작품소개
----------------------------------------
[19금][능욕물][하렘물?][서스펜스물]

얻은 자의 인생을 바꾸어준다는 이야기가 떠도는 어플 초이스.
우연찮게 그것을 얻게된 염세적 인간 승훈의 야릇한 인생역전기입니다.

에로틱하면서도 노골적으로 이야기를 이어갈테니 부디 즐겁게 봐주세요!!
----------------------------------------
이미지 : //cdn1.munpia.com/files/attach/2016/0517/003/d3owjwOStQN8jLLH.jpg
제목 : 어플초이스~능욕의 연쇄~
장르 : 현대판타지,  게임
연령등급 : 19세
완결여부 : 완결
작가 : 
권하율


작품등록일 : 2016.05.02 17:55
총 연재수 : 100 회
출판사: 정보 없음


작품소개
----------------------------------------
[19금][능욕물][하렘물?][서스펜스물]

얻은 자의 인생을 바꾸어준다는 이야기가 떠도는 어플 초이스.
우연찮게 그것을 얻게된 염세적 인간 승훈의 야릇한 인생역전기입니다.

에로틱하면서도 노골적으로 이야기를 이어갈테니 부디 즐겁게 봐주세요!!
----------------------------------------
이미지 : //cdn1.munpia.com/files/attach/2016/0517/003/d3owjwOStQN8jLLH.jpg
제목 : 어플초이스~능욕의 연쇄~
장르 : 현대판타지,  게임
연령등급 : 19세
완결여부 : 완결
작가 : 
권하율


작품등록일 : 2016.05.02 17:55
총 연재수 : 100 회
출판사: 정보 없음


작품소개
----------------------------------------
[19금][능욕물][하렘물?][서스펜스물]

얻은 자의 인생을 바꾸어준다는 이야기가 떠도는 어플 초이스.
우연찮게 그것을 얻게된 염세적 인간 승훈의 야릇한 인생역전기입니다.

에로틱하면서도 노골적으로 이야기를 이어갈



작품소개
----------------------------------------
[19금][능욕물][하렘물?][서스펜스물]

얻은 자의 인생을 바꾸어준다는 이야기가 떠도는 어플 초이스.
우연찮게 그것을 얻게된 염세적 인간 승훈의 야릇한 인생역전기입니다.

에로틱하면서도 노골적으로 이야기를 이어갈테니 부디 즐겁게 봐주세요!!
----------------------------------------
이미지 : //cdn1.munpia.com/files/attach/2016/0517/003/d3owjwOStQN8jLLH.jpg
제목 : 어플초이스~능욕의 연쇄~
장르 : 현대판타지,  게임
연령등급 : 19세
완결여부 : 완결
작가 : 
권하율


작품등록일 : 2016.05.02 17:55
총 연재수 : 100 회
출판사: 정보 없음


작품소개
----------------------------------------
[19금][능욕물][하렘물?][서스펜스물]

얻은 자의 인생을 바꾸어준다는 이야기가 떠도는 어플 초이스.
우연찮게 그것을 얻게된 염세적 인간 승훈의 야릇한 인생역전기입니다.

에로틱하면서도 노골적으로 이야기를 이어갈테니 부디 즐겁게 봐주세요!!
----------------------------------------
이미지 : //cdn1.munpia.com/files/attach/2016/0517/003/d3owjwOStQN8jLLH.jpg
제목 : 어플초이스~능욕의 연쇄~
장르 : 현대판타지,  게임
연령등급 : 19세
완결여부 : 완결
작가 : 
권하율


작품등록일 : 2016.05.02 17:55
총 연재수 : 100 회
출판사: 정보 없음


작품소개
----------------------------------------
[19금][능욕물][하렘물?][서스펜스물]

얻은 자의 인생을 바꾸어준다는 이야기가 떠도는 어플 초이스.
우연찮게 그것을 얻게된 염세적 인간 승훈의 야릇한 인생역전기입니다.

에로틱하면서도 노골적으로 이야기를 이어갈

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=85.0.4183.83)
